In [ ]:
from img_classify.dataset import *
from img_classify.downloader import *
from img_classify.evaluate_model import *
from img_classify.tools import *
from keras import Sequential
from keras.applications import *
from keras.callbacks import *
from keras.losses import *
from keras.optimizers import AdamW
from keras.src.utils import set_random_seed
from keras.layers import *
from datetime import datetime

In [ ]:
# Tải dữ liệu từ thư viện Kaggle
if not os.path.exists('datasets/covid_19'):
	kaggle_downloader(
		'covid_19',
		'plameneduardo/sarscov2-ctscan-dataset'
	)

In [ ]:

# Thiết lập chung
train_path = 'datasets/covid_19/'
img_save_path = 'imgs/covid_19/'
set_random_seed(69)
IMG_SIZE = (192, 192, 3)


In [ ]:
# Tạo nhãn cho tập dữ liệu
class_names = create_label_from_dir(train_path)

In [ ]:
# Kiểm tra ảnh trong thư mục
check_dir(train_path)

In [ ]:
# Kiểm tra độ cân bằng dữ liệu
check_balance(
	train_path,
	class_names,
	img_save_path=os.path.join(img_save_path, 'check_balance.jpg')
)

In [ ]:
# Tái cân bằng nhãn tập dữ liệu
img_model = ImageDataGenerator(
	horizontal_flip=True,
	zoom_range=.02
)

fix_imbalance_with_image_augmentation(
	train_path,
	img_size=(IMG_SIZE[0], IMG_SIZE[1]),
	img_model=img_model,
	class_names=class_names
)

In [ ]:
## Nạp ảnh
images, labels = images_to_array(train_path, class_names, (IMG_SIZE[0], IMG_SIZE[1]))

## Tách mảng để Train/Test/Val (70/25/5)
(train_images, train_labels), (test_images, test_labels), (val_images, val_labels) = train_test_val_split(
	images,
	labels,
	train_size=.70,
	test_size=.25,
	val_size=.05
)

## Tăng cường ảnh tập Train
# train_img_model = ImageDataGenerator(
# 	horizontal_flip=True,
# 	brightness_range=(.3, .7),
# 	zoom_range=.2,
# 	shear_range=.2,
# 	height_shift_range=.1
# )
# train_img_model.flow(
# 	train_images,
# 	train_labels,
# 	seed=69
# )

## Rescale ảnh
train_images = train_images / 255.
test_images = test_images / 255.
val_images = val_images / 255.

In [ ]:
# Xây dựng mô hình
def my_model():
	#  Thiết lập mô hình
	#base_model = MobileNetV2(include_top=False, input_shape=IMG_SIZE)
	#base_model.trainable = False

	model = Sequential(
		[
			#base_model,

			InputLayer(input_shape=IMG_SIZE),

			Conv2D(32, kernel_size=3, strides=2, padding='same', activation='relu'),
			Dropout(.10),
			Conv2D(32, kernel_size=3, strides=2, padding='same', activation='relu'),
			Dropout(.10),
			Conv2D(32, kernel_size=3, strides=2, padding='same', activation='relu'),
			Dropout(.10),

			Conv2D(64, kernel_size=3, strides=2, padding='same', activation='relu'),
			Dropout(.10),
			Conv2D(64, kernel_size=3, strides=2, padding='same', activation='relu'),
			Dropout(.10),
			Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu'),
			Dropout(.10),

			Conv2D(128, kernel_size=3, strides=2, padding='same', activation='relu'),
			Dropout(.10),
			Conv2D(128, kernel_size=3, strides=2, padding='same', activation='relu'),
			Dropout(.10),
			Conv2D(128, kernel_size=3, strides=2, padding='same', activation='relu'),
			Dropout(.10),

			GlobalAveragePooling2D(),

			Flatten(),
			Dense(256, kernel_regularizer='l2', activation='relu'),
			Dense(32, activation='relu'),
			Dense(len(class_names), activation='sigmoid')
		]
	)

	# Biên dịch mô hình
	model.compile(
		AdamW(),
		BinaryCrossentropy(),
		['accuracy']
	)
	return model

# Chọn mô hình
model = my_model()
# Khái quát mô hình
model.summary()
# Đào tạo mô hình
history = model.fit(
	train_images,
	train_labels,
	epochs=10,
	validation_data=(test_images, test_labels),
	callbacks=[
		EarlyStopping(
			monitor='val_accuracy',
			patience=5,
			restore_best_weights=True
		),
		ReduceLROnPlateau(
			monitor='val_accuracy',
			factor=.1,
			patience=1,
			min_lr=.0001
		),
		ModelCheckpoint(
			f'models/checkpoints/covid_19/{datetime.now().strftime("%m%d%Y%H%M%S")}/',
			monitor='val_loss',
			save_best_only=True
		)
	]
)
# Lưu mô hình
model.save('models/covid_19_model.keras')
## Best result: 94%

In [ ]:
# Đánh giá quá trình đào tạo
EvalofTraining(history, img_save_path)
# Đánh giá mô hình qua tập Test
sai_so, do_chinh_xac = model.evaluate(test_images, test_labels)
print(f'Sai số: {sai_so}')
print(f'Độ chính xác: {do_chinh_xac}')
# Dự đoán để đánh giá mô hình
pred_labels = model.predict(val_images)
eval_of_model_with_images(
	3, 3, val_images, pred_labels, val_labels, class_names,
	img_save_path=os.path.join(img_save_path, 'eval_of_model_with_images.jpg')
)
heatmap_plot(
	val_labels, pred_labels, class_names,
	img_save_path=os.path.join(img_save_path, 'eval_of_model_with_heatmap.jpg')
)